# <div style='color:#fe618e;font-weight:800;'>optuna库选择超参数</div>


In [1]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import time
from matplotlib import pyplot as plt
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import VecFrameStack
import os
from stable_baselines3 import PPO

from stable_baselines3.common.results_plotter import load_results, ts2xy
import numpy as np
from stable_baselines3.common.callbacks import BaseCallback

import optuna
from stable_baselines3.common.evaluation import evaluate_policy



c:\ProgramData\Anaconda3\envs\RL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

monitor_dir = r'./monitor_log/'
os.makedirs(monitor_dir,exist_ok=True)
env = Monitor(env,monitor_dir)

env = GrayScaleObservation(env,keep_dim=True)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env,4,channels_order='last')


In [2]:
def optimize_ppo(trial): 
    return {
        'n_steps':trial.suggest_int('n_steps', 2048, 8192),
        'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999),
        'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
        'clip_range':trial.suggest_uniform('clip_range', 0.1, 0.4),
        'gae_lambda':trial.suggest_uniform('gae_lambda', 0.8, 0.99)
    }

In [4]:
# from stable_baselines3.common.evaluation import evaluate_policy


In [3]:
def optimize_function(trial):
    
    try:
        env = gym_super_mario_bros.make('SuperMarioBros-v0')
        env = JoypadSpace(env, SIMPLE_MOVEMENT)

        monitor_dir = r'./monitor_log/'
        os.makedirs(monitor_dir,exist_ok=True)
        env = Monitor(env,monitor_dir)

        env = GrayScaleObservation(env,keep_dim=True)
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env,4,channels_order='last')


        model_params = optimize_ppo(trial) 
        # model_params = {
        #     'n_steps':trial.suggest_int('n_steps', 2048, 8192),
        #     'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999),
        #     'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
        #     'clip_range':trial.suggest_uniform('clip_range', 0.1, 0.4),
        #     'gae_lambda':trial.suggest_uniform('gae_lambda', 0.8, 0.99)
        # }

        tensorboard_log = r'./tensorboard_log/'
        model = PPO("CnnPolicy", env, verbose=0,tensorboard_log=tensorboard_log,**model_params)
        model.learn(total_timesteps=1000)
        # model.learn(total_timesteps=200000)
    
        # mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
        mean_reward, _ = evaluate_policy(model, env,n_eval_episodes=1)
    
    
        env.close()
    
        save_model_dir = r'F:\\RL_Mario1\\'
        os.makedirs(save_model_dir,exist_ok=True)
        SAVE_PATH = os.path.join(save_model_dir, 'trial_{}_best_model'.format(trial.number))
        model.save(SAVE_PATH)
    
        return mean_reward    

    except Exception as e:
        return -1000
    
    

In [6]:
# import optuna

In [4]:
study = optuna.create_study(direction='maximize')
# study.optimize(optimize_function, n_trials=100)
study.optimize(optimize_function, n_trials=3)

[I 2023-04-22 11:57:33,114] A new study created in memory with name: no-name-b1a4afdb-5648-4417-a1f1-b34e799c7b30
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14148\1618903399.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999),
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14148\1618903399.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14148\1618903399.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in 

In [5]:
study.best_params

{'n_steps': 7555,
 'gamma': 0.8755316882160081,
 'learning_rate': 6.317305306993189e-05,
 'clip_range': 0.13613525124152984,
 'gae_lambda': 0.9825104568433543}

In [6]:
study.best_trial

FrozenTrial(number=0, state=TrialState.COMPLETE, values=[-1000.0], datetime_start=datetime.datetime(2023, 4, 22, 11, 57, 33, 115461), datetime_complete=datetime.datetime(2023, 4, 22, 11, 59, 50, 869852), params={'n_steps': 7555, 'gamma': 0.8755316882160081, 'learning_rate': 6.317305306993189e-05, 'clip_range': 0.13613525124152984, 'gae_lambda': 0.9825104568433543}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_steps': IntDistribution(high=8192, log=False, low=2048, step=1), 'gamma': FloatDistribution(high=0.9999, log=True, low=0.8, step=None), 'learning_rate': FloatDistribution(high=0.0001, log=True, low=1e-05, step=None), 'clip_range': FloatDistribution(high=0.4, log=False, low=0.1, step=None), 'gae_lambda': FloatDistribution(high=0.99, log=False, low=0.8, step=None)}, trial_id=0, value=None)